### Reading in the data

In [7]:
import pandas as pd
import numpy as np
from scipy import stats

ims = pd.read_csv('IMS_MEDICINES.csv', encoding = "ISO-8859-1", sep = ";")
ims.head()

,Ind,Pack,Region,Channel,Pack Molecule String,Pack Form,Pack Size,Pack Strength,Pack Volume,Product,...,Pack Price - QTR3 2016,Pack Price - QTR4 2016,Pack Price - QTR1 2015,Pack Price - QTR2 2015,Pack Price - QTR3 2015,Pack Price - QTR4 2015,Pack Price - QTR1 2014,Pack Price - QTR2 2014,Pack Price - QTR3 2014,Pack Price - QTR4 2014
0,1,KETOSTERIL TAB FILM CTD 600 MG 100,Manila,Retail,(RS)-3 METHYL-2-OXOVALERIANIC ACID+(RS)-3-METH...,TAB FILM CTD,100,600 MG,0,KETOSTERIL - [FK2],...,3528,3528,3528,3528,3528,3528,3528,3528,3528,3528
1,2,RENOKET TAB FILM CTD 100,Manila,Retail,(RS)-3 METHYL-2-OXOVALERIANIC ACID+(RS)-3-METH...,TAB FILM CTD,100,0,0,RENOKET - [MRE],...,2589.29,2641.07,2589.29,2589.29,2589.29,2589.29,0,0,0,0
2,3,KETOSTERIL TAB FILM CTD 600 MG 100,Manila,Hospital,(RS)-3 METHYL-2-OXOVALERIANIC ACID+(RS)-3-METH...,TAB FILM CTD,100,600 MG,0,KETOSTERIL - [FK2],...,3528,3528,3528,3528,3528,3528,3528,3528,3528,3528
3,4,RENOKET TAB FILM CTD 100,Manila,Hospital,(RS)-3 METHYL-2-OXOVALERIANIC ACID+(RS)-3-METH...,TAB FILM CTD,100,0,0,RENOKET - [MRE],...,2589.29,2641.07,2589.29,2589.29,2589.29,2589.29,0,0,0,0
4,5,KETOSTERIL TAB FILM CTD 600 MG 100,Luzon,Retail,(RS)-3 METHYL-2-OXOVALERIANIC ACID+(RS)-3-METH...,TAB FILM CTD,100,600 MG,0,KETOSTERIL - [FK2],...,3528,3528,3528,3528,3528,3528,3528,3528,3528,3528


In [8]:
ims.columns.values

array(['Ind', 'Pack', 'Region', 'Channel', 'Pack Molecule String',
       'Pack Form', 'Pack Size', 'Pack Strength', 'Pack Volume', 'Product',
       'Ethical Status', 'Manufacturer', 'License Type (MIDAS)',
       'Pack Launch Date', 'Drug Flag', 'ATC1', 'ATC2', 'ATC3', 'ATC4',
       'Values - MAT 2016/12', 'Values - MAT 2015/12',
       'Values - MAT 2014/12', 'Values - QTR1 2016', 'Values - QTR2 2016',
       'Values - QTR3 2016', 'Values - QTR4 2016', 'Values - QTR1 2015',
       'Values - QTR2 2015', 'Values - QTR3 2015', 'Values - QTR4 2015',
       'Values - QTR1 2014', 'Values - QTR2 2014', 'Values - QTR3 2014',
       'Values - QTR4 2014', 'Counting Units - MAT 2016/12',
       'Counting Units - MAT 2015/12', 'Counting Units - MAT 2014/12',
       'Counting Units - QTR1 2016', 'Counting Units - QTR2 2016',
       'Counting Units - QTR3 2016', 'Counting Units - QTR4 2016',
       'Counting Units - QTR1 2015', 'Counting Units - QTR2 2015',
       'Counting Units - QTR3 2015', '

### Looking for comparable tuples of drugs

In [34]:
tuples = pd.DataFrame(columns = ims.columns.values)
tuples['tup'] = 0 #create a new df for holding the tuples
tuple_counter = 0 #index to denote each tuple (will be the same for each tuple)

for drug in ims['Pack Molecule String'].unique():
    sub = ims.loc[ims['Pack Molecule String'] == drug]

    while len(sub.index) > 0:
        ref = sub.iloc[0]
        ref['tup'] = tuple_counter
    
        temp_tuples = pd.DataFrame(columns = tuples.columns.values)
        temp_tuples = temp_tuples.append(ref)
    
        sub = sub.loc[sub['Ind'] != ref['Ind']]
    
        for index, row in sub.iterrows():
            if row['Pack Form'] == ref['Pack Form'] and row['Pack Strength'] == ref['Pack Strength'] and row['Pack Volume'] == ref['Pack Volume']:
                row['tup'] = tuple_counter
                temp_tuples = temp_tuples.append(row)
                sub = sub.loc[sub['Ind'] != row['Ind']]
    
        tuple_counter += 1
    
        if len(temp_tuples.index) > 1:
            tuples = tuples.append(temp_tuples)

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\asus\Anaconda3\lib\site-packages\pandas\core\series.py:747: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value


In [35]:
tuples.to_csv('tuples (without pack size).csv')

In [ ]:
#loading tuples from a checkpoint
tuples_wps = pd.read_csv('tuples (without pack size).csv', encoding = "ISO-8859-1", sep = ";")

In [39]:
tuples.dropna(subset = ['Price per Unit - MAT 2016/12']) #scrape off missing values
tuples = tuples.loc[tuples['Drug Flag'] != 'Non Drug'] #disregard non-drugs

### Searching for price variations among molecules, between licenses (originator, branded non-originator, and unbranded non-originator)

In [ ]:
#description of each tuple
tups = tuples['tup'].unique()
pack_molecule = []
pack_form = []
pack_strength = []
with_gen_equivalent = []
d_orig_brand = []
d_brand_gen = []
d_orig_gen = []

for tup in tups:
    sub = tuples.iloc[tuples['tup'] == tup]
    
    ref = sub.iloc[0]
    pack_molecule.append(ref['Pack Molecule String'])
    pack_form.append(ref['Pack Form'])
    pack_strength.append(ref['Pack Strength'])
    
    lics = sub['License (MIDAS)'].unique()
        

from scipy.stats import variation

tuple_indices = tuples['tup'].unique().tolist()
no_tuples = []
cv = []

for i in tuple_indices:
    sub = tuples.loc[tuples['tup'] == i]
    no_tuples.append(len(sub.index))
    if len(sub.index) > 3:
        cv.append(variation(sub['Price per Unit - MAT 2016/12'], nan_policy = 'omit'))
    else:
        cv.append(np.nan)

cvs = dict(zip(no_tuples, tuple_indices, cv))

In [43]:
tuples.head()

,Ind,Pack,Region,Channel,Pack Molecule String,Pack Form,Pack Size,Pack Strength,Pack Volume,Product,...,Pack Price - QTR4 2016,Pack Price - QTR1 2015,Pack Price - QTR2 2015,Pack Price - QTR3 2015,Pack Price - QTR4 2015,Pack Price - QTR1 2014,Pack Price - QTR2 2014,Pack Price - QTR3 2014,Pack Price - QTR4 2014,tup
0,1,KETOSTERIL TAB FILM CTD 600 MG 100,Manila,Retail,(RS)-3 METHYL-2-OXOVALERIANIC ACID+(RS)-3-METH...,TAB FILM CTD,100,600 MG,0,KETOSTERIL - [FK2],...,3528,3528,3528,3528,3528,3528,3528,3528,3528,0
2,3,KETOSTERIL TAB FILM CTD 600 MG 100,Manila,Hospital,(RS)-3 METHYL-2-OXOVALERIANIC ACID+(RS)-3-METH...,TAB FILM CTD,100,600 MG,0,KETOSTERIL - [FK2],...,3528,3528,3528,3528,3528,3528,3528,3528,3528,0
4,5,KETOSTERIL TAB FILM CTD 600 MG 100,Luzon,Retail,(RS)-3 METHYL-2-OXOVALERIANIC ACID+(RS)-3-METH...,TAB FILM CTD,100,600 MG,0,KETOSTERIL - [FK2],...,3528,3528,3528,3528,3528,3528,3528,3528,3528,0
6,7,KETOSTERIL TAB FILM CTD 600 MG 100,Luzon,Hospital,(RS)-3 METHYL-2-OXOVALERIANIC ACID+(RS)-3-METH...,TAB FILM CTD,100,600 MG,0,KETOSTERIL - [FK2],...,3528,3528,3528,3528,3528,3528,3528,3528,3528,0
8,9,KETOSTERIL TAB FILM CTD 600 MG 100,Visayas,Retail,(RS)-3 METHYL-2-OXOVALERIANIC ACID+(RS)-3-METH...,TAB FILM CTD,100,600 MG,0,KETOSTERIL - [FK2],...,"3,528.00","3,528.00","3,528.00","3,528.00","3,528.00","3,528.00","3,528.00","3,528.00","3,528.00",0
